In [95]:
#from IPython.core.magic import register_line_magic

#@register_line_magic
##3def pip(args):
    #"""Use pip from the current kernel"""
    #from pip import main
    #main(args.split())

In [96]:
##%pip install yahoo_fin --upgrade
##%pip install requests_html
#!pip uninstall -y yfinance
#%pip install yfinance --upgrade --no-cache-dir
#pip install get-all-tickers
#%pip install reticker

In [97]:
import os
import pandas as pd
import numpy as np
import datetime as dt
##import yfinance as yf
import re
import requests
from bs4 import BeautifulSoup
##import reticker
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [98]:
##data = yf.download(tickers = "gme", period="2y", interval = "1h")
##data.head()

## for i in df["title"][:20]:
    ##print(re.findall(r'[$][A-Za-z][\S]*', str(i)))

##extractor = reticker.TickerExtractor()

##for i in df["title"][:20]:
    ##print(extractor.extract(i))

In [99]:
df=pd.read_csv("reddit_wsb.csv")
df.head()

,title,score,id,url,comms_num,created,body,timestamp
0,"It's not about the money, it's about sending a...",55,l6ulcx,https://v.redd.it/6j75regs72e61,6,1.611863e+09,NaN,2021-01-28 21:37:41
1,Math Professor Scott Steiner says the numbers ...,110,l6uibd,https://v.redd.it/ah50lyny62e61,23,1.611862e+09,NaN,2021-01-28 21:32:10
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,l6ugk6,https://sec.report/Document/0001193125-21-019848/,74,1.611862e+09,NaN,2021-01-28 21:28:57
4,"Not to distract from GME, just thought our AMC...",71,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,156,1.611862e+09,NaN,2021-01-28 21:26:56


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53187 entries, 0 to 53186
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      53187 non-null  object 
 1   score      53187 non-null  int64  
 2   id         53187 non-null  object 
 3   url        53187 non-null  object 
 4   comms_num  53187 non-null  int64  
 5   created    53187 non-null  float64
 6   body       24738 non-null  object 
 7   timestamp  53187 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 3.2+ MB


In [101]:
#convert date column to datetime 
df['timestamp'] = pd.to_datetime(df['timestamp'], 
                                    format='%Y-%m-%dT%H:%M:%S.%f')
df.dtypes

title                object
score                 int64
id                   object
url                  object
comms_num             int64
created             float64
body                 object
timestamp    datetime64[ns]
dtype: object

In [102]:
##creating year, mont, dayofweek, hour, min, second columns for eda
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.dayofweek
df['hour'] = df['timestamp'].dt.hour
df['min'] = df['timestamp'].dt.minute
df['sec'] = df['timestamp'].dt.second

##we can get name of the days if we need to.
##dayoftheweek = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
##days = list(dayoftheweek.values())
##df['day'] = df['timestamp'].apply(lambda x : dayoftheweek[x.weekday()])

In [103]:
df["body"] = df["body"].fillna("")##fill nan's with empty
df["original_text"] = df["title"] + ' ' + df["body"]##concatane body and title to create a new column called text
df = df.drop(columns = ["body", "title", "id", "url", "created"])##drop unrelated columns

In [104]:
df.head()

,score,comms_num,timestamp,year,month,day,hour,min,sec,original_text
0,55,6,2021-01-28 21:37:41,2021,1,3,21,37,41,"It's not about the money, it's about sending a..."
1,110,23,2021-01-28 21:32:10,2021,1,3,21,32,10,Math Professor Scott Steiner says the numbers ...
2,0,47,2021-01-28 21:30:35,2021,1,3,21,30,35,Exit the system The CEO of NASDAQ pushed to ha...
3,29,74,2021-01-28 21:28:57,2021,1,3,21,28,57,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...
4,71,156,2021-01-28 21:26:56,2021,1,3,21,26,56,"Not to distract from GME, just thought our AMC..."


In [105]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
##https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python

In [108]:
def clean_text_date(text):
    text = text.lower()
    
    # remove emojis
    text = text.encode('ascii','ignore')
    text = text.decode()

    # replacing handlers with empty string
    text = re.sub('@[^\s]+',"",text)
    
    # remove punctuations and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)
    
    # replacing URLs with empty string
    text = re.sub(r"http\S+", "",text)

    # remove all the special characters
    text = ' '.join(re.findall(r'\w+', text))

    # replacing single characters with empty string
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    # substituting multiple spaces with single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    
    # remove stopwords
    text = ' '.join([word for word in text.split() if not word in set(stopwords.words('english'))])
    
    # words to singular forms
    lem = WordNetLemmatizer()
    text = lem.lemmatize(text)
    
    return text

In [109]:
# Text Preprocessing
df["text"] = df["original_text"].apply(lambda x : clean_text_date(x))

In [110]:
df.head()

,score,comms_num,timestamp,year,month,day,hour,min,sec,original_text,text
0,55,6,2021-01-28 21:37:41,2021,1,3,21,37,41,"It's not about the money, it's about sending a...",money sending message
1,110,23,2021-01-28 21:32:10,2021,1,3,21,32,10,Math Professor Scott Steiner says the numbers ...,math professor scott steiner says numbers spel...
2,0,47,2021-01-28 21:30:35,2021,1,3,21,30,35,Exit the system The CEO of NASDAQ pushed to ha...,exit system ceo nasdaq pushed halt trading giv...
3,29,74,2021-01-28 21:28:57,2021,1,3,21,28,57,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,new sec filing gme someone less retarded pleas...
4,71,156,2021-01-28 21:26:56,2021,1,3,21,26,56,"Not to distract from GME, just thought our AMC...",distract gme thought amc brothers aware
